# Project Solution Lab 2: 

# Problema: Predicción de retrasos en aviones

Los objetivos múltiples de este bloc de notas son:
- Procesar y crear un conjunto de datos a partir de archivos .zip descargados
- Analizar datos exploratorios (EDA)
- Establecer un modelo de referencia
- Pasar de un modelo simple a un modelo de conjunto
- Optimización de hiperparámetros
- Comprobar la importancia de las características

## Introducción al escenario empresarial
Trabajas en un sitio web de reserva de viajes que está intentando mejorar la experiencia del cliente en vuelos que se han retrasado. La compañía quiere crear una característica para que los clientes sepan si el vuelo se retrasará debido al clima cuando reserven un vuelo hacia o desde los aeropuertos más concurridos para viajes nacionales en los Estados Unidos. 

Se te encarga que resuelvas parte de este problema mediante el uso del machine learning para identificar si el vuelo se retrasará debido al clima. Se te ha dado acceso a un conjunto de datos sobre el rendimiento de la puntualidad de los vuelos nacionales operados por grandes compañías aéreas. Puedes utilizar estos datos para entrenar un modelo de machine learning y predecir si el vuelo se retrasará en los aeropuertos más concurridos.

## Acerca de este conjunto de datos
Este conjunto de datos contiene las horas de salida y llegada programadas y reales reportadas por compañías aéreas certificadas de Estados Unidos que representan al menos el 1 % de los ingresos nacionales de pasajeros regulares. Los datos fueron recogidos por la Oficina de Información Aérea de la Oficina de Estadísticas del Transporte (BTS). El conjunto de datos contiene fecha, hora, origen, destino, aerolínea, distancia y estado de retraso de los vuelos entre 2013 y 2018.

### Características
Para obtener más información acerca de las características del conjunto de datos, consulta [Características del conjunto de datos de demoras en la puntualidad] https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ.

### Atribuciones del conjunto de datos  

Sitio web: https://www.transtats.bts.gov/

# Paso 1: Formulación de problemas y recopilación de datos

Para empezar el proyecto, escribe a continuación algunas frases que resuman la cuestión y el objetivo de negocio que intentas lograr en este escenario. Incluye la métrica empresarial a la que quieres que aspire tu equipo. Tras definir esta información, escribe claramente el enunciado del problema de machine learning. Por último, añade uno o dos comentarios sobre el tipo de machine learning que esto representa. 

#### <span style="color: blue;">Presentación del proyecto: incluye un resumen de estos detalles en las presentaciones del proyecto.</span>


### 1. Determina si es apropiado desplegar ML y los motivos para hacerlo.
\# Escribe la respuesta aquí

### 2. Formula la cuestión del negocio, las métricas correctas y el resultado que deseas que genere el machine learning.
\# Escribe la respuesta aquí

### 3. Identifica el tipo de problema de ML.
\# Escribe la respuesta aquí

### 4. Analiza la idoneidad de los datos con los que estás trabajando.
\# Escribe la respuesta aquí

### Configuración

Ahora que hemos decidido dónde concentrarnos, preparemos todo para que puedas empezar a trabajar en la resolución del problema.

**Nota:** Este bloc de notas se ha creado y probado en una instancia del bloc de notas `ml.m4.xlarge`. 

Reemplaza **`<LabBucketName>`** por el nombre de recurso que se proporcionó con tu cuenta de laboratorio.

In [1]:
# Change the bucket according to your information
bucket = '<LabBucketName>'

In [2]:
%%capture 

%matplotlib inline
%pip install --upgrade boto3 botocore===1.33.2 -q
%pip install xgboost=='0.90' -q

In [3]:
import os
from pathlib2 import Path
from zipfile import ZipFile
import time

import pandas as pd
import numpy as np
import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

# Paso 2: Preprocesamiento y visualización de datos
En esta fase del preprocesamiento de los datos, debes aprovechar la oportunidad para explorar y visualizar tus datos para conocerlos mejor. En primer lugar, importa las bibliotecas necesarias y lee los datos en un DataFrame de Pandas. Luego, explora tus datos. Busca la forma del conjunto de datos y explora las columnas y los tipos de columnas con las que vas a trabajar (numérica, categórica). Considera la posibilidad de obtener estadísticas básicas de las características para hacerte una idea de las medias y rangos de las características. Examina detenidamente la columna de destino y determina su distribución.

### Preguntas específicas a considerar
1. ¿Qué puedes deducir de las estadísticas básicas que ejecutaste en las características? 

2. ¿Qué puedes deducir de las distribuciones de las clases de destino?

3. ¿Has deducido algo más de la exploración de los datos?

#### <span style="color: blue;">Presentación del proyecto: incluye en las presentaciones del proyecto<b> </b>un resumen de tus respuestas a estas preguntas, y a otras similares.</span>

Comienza incorporando el conjunto de datos desde un bucket público de Amazon S3 a este entorno de bloc de notas.

In [4]:
# Check whether the file is already in the desired path or if it needs to be downloaded

base_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
csv_base_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays/'
file_path = 'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_1.zip'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['mkdir', '-p', csv_base_path])
    subprocess.run(['aws', 's3', 'cp', 
                    's3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/csvFlightData-5/', 
                    base_path,'--recursive'])
else:
    print('File already downloaded!')

File already downloaded!


In [5]:
zip_files = [str(file) for file in list(Path(base_path).iterdir()) if '.zip' in str(file)]
len(zip_files)

60

#### Extraer archivos CSV de archivos ZIP

In [6]:
def zip2csv(zipFile_name , file_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays'):
    """
    Extract csv from zip files
    zipFile_name: name of the zip file
    file_path : name of the folder to store csv
    """
    try:
        with ZipFile(zipFile_name, 'r') as z: 
            print(f'Extracting {zipFile_name} ') 
            z.extractall(path=file_path) 
    except:
        print(f'zip2csv failed for {zipFile_name}')

for file in zip_files:
    zip2csv(file)

print("Files Extracted")

Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_7.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_3.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_5.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_5.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2016_5.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2018_12.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_9.zip 
Extracting /home/ec2-user/SageMaker/project/data/FlightDelays/On_Tim

In [7]:
csv_files = [str(file) for file in list(Path(csv_base_path).iterdir()) if '.csv' in str(file)]
len(csv_files)

60

Antes de cargar el archivo CSV, lee el archivo HTML de la carpeta extraída. Este archivo HTML incluye el contexto y más información acerca de las características que se incluyen en el conjunto de datos.

In [8]:
from IPython.display import IFrame, HTML

IFrame(src="../project/data/csvFlightDelays/readme.html", width=1000, height=600)

#### Cargar CSV de muestra

Antes de combinar todos los archivos CSV, familiarícese con los datos de un único archivo CSV. Con Pandas, lee primero el archivo `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv`. Puedes usar la función de Python integrada `read_csv` ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)).

In [9]:
df_temp = pd.read_csv(f"{csv_base_path}On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv")

**Pregunta**: imprime la longitud de fila y columna en el conjunto de datos e imprime los nombres de las columnas.

**Sugerencia**: utiliza la función `<dataframe>.shape` para ver las filas y columnas de un DataFrame y `<dataframe>.columns` para ver los nombres de las columnas.

In [10]:
df_shape = df_temp.shape
print(f'Rows and columns in one csv file is {df_shape}')

Rows and columns in one csv file is (585749, 110)


**Pregunta**: imprime las primeras 10 filas del conjunto de datos.  

**Sugerencia**: utiliza la función integrada de Pandas `head (x)` para imprimir `x` número de filas.

In [11]:
df_temp.head(10)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2018,3,9,3,1,2018-09-03,9E,20363,9E,N908XJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,3,9,9,7,2018-09-09,9E,20363,9E,N315PQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,3,9,10,1,2018-09-10,9E,20363,9E,N582CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,3,9,13,4,2018-09-13,9E,20363,9E,N292PQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,3,9,14,5,2018-09-14,9E,20363,9E,N600LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018,3,9,16,7,2018-09-16,9E,20363,9E,N316PQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018,3,9,17,1,2018-09-17,9E,20363,9E,N916XJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018,3,9,20,4,2018-09-20,9E,20363,9E,N371CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,3,9,21,5,2018-09-21,9E,20363,9E,N601LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018,3,9,23,7,2018-09-23,9E,20363,9E,N906XJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Pregunta**: imprime todas las columnas del conjunto de datos. Utiliza `<dataframe>.columns` para ver los nombres de las columnas.

In [12]:
print(f'The column names are :')
print('#########')
for col in df_temp.columns:
    print(col)

The column names are :
#########
Year
Quarter
Month
DayofMonth
DayOfWeek
FlightDate
Reporting_Airline
DOT_ID_Reporting_Airline
IATA_CODE_Reporting_Airline
Tail_Number
Flight_Number_Reporting_Airline
OriginAirportID
OriginAirportSeqID
OriginCityMarketID
Origin
OriginCityName
OriginState
OriginStateFips
OriginStateName
OriginWac
DestAirportID
DestAirportSeqID
DestCityMarketID
Dest
DestCityName
DestState
DestStateFips
DestStateName
DestWac
CRSDepTime
DepTime
DepDelay
DepDelayMinutes
DepDel15
DepartureDelayGroups
DepTimeBlk
TaxiOut
WheelsOff
WheelsOn
TaxiIn
CRSArrTime
ArrTime
ArrDelay
ArrDelayMinutes
ArrDel15
ArrivalDelayGroups
ArrTimeBlk
Cancelled
CancellationCode
Diverted
CRSElapsedTime
ActualElapsedTime
AirTime
Flights
Distance
DistanceGroup
CarrierDelay
WeatherDelay
NASDelay
SecurityDelay
LateAircraftDelay
FirstDepTime
TotalAddGTime
LongestAddGTime
DivAirportLandings
DivReachedDest
DivActualElapsedTime
DivArrDelay
DivDistance
Div1Airport
Div1AirportID
Div1AirportSeqID
Div1WheelsOn
Div1

**Pregunta**: imprime todas las columnas del conjunto de datos que contienen la palabra 'Del'. Esto ayudará a ver cuántas columnas tienen datos de demoras en ellas.

**Sugerencia**: puedes usar una comprensión de listas de Python que incluya valores que superen ciertos criterios de declaración `if`.

Por ejemplo: `[x for x in [1,2,3,4,5] if x > 2]`  

**Sugerencia**: puedes usar la palabra clave `in` ([documentación](https://www.w3schools.com/python/ref_keyword_in.asp)) para comprobar si el valor está en una lista. 

Por ejemplo: `5 in [1,2,3,4,5]`

In [13]:
[col for col in df_temp.columns if 'Del' in col]

['DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'DivArrDelay']

Estas son otras preguntas que te ayudarán a obtener más información sobre el conjunto de datos.

**Preguntas**   
1. ¿Cuántas filas y columnas tiene el conjunto de datos?   
2. ¿Cuántos años se incluyen en el conjunto de datos?   
3. ¿Cuál es el rango de fechas para el conjunto de datos?   
4. ¿Qué aerolíneas se incluyen en el conjunto de datos?   
5. ¿Qué aeropuertos de origen y destino están incluidos?

In [14]:
print("The #rows and #columns are ", df_temp.shape[0] , " and ", df_temp.shape[1])
print("The years in this dataset are: ", list(df_temp.Year.unique()))
print("The months covered in this dataset are: ", sorted(list(df_temp.Month.unique())))
print("The date range for data is :" , min(df_temp.FlightDate), " to ", max(df_temp.FlightDate))
print("The airlines covered in this dataset are: ", list(df_temp.Reporting_Airline.unique()))
print("The Origin airports covered are: ", list(df_temp.Origin.unique()))
print("The Destination airports covered are: ", list(df_temp.Dest.unique()))

The #rows and #columns are  585749  and  110
The years in this dataset are:  [2018]
The months covered in this dataset are:  [9]
The date range for data is : 2018-09-01  to  2018-09-30
The airlines covered in this dataset are:  ['9E', 'B6', 'WN', 'YV', 'YX', 'EV', 'AA', 'AS', 'DL', 'HA', 'UA', 'F9', 'G4', 'MQ', 'NK', 'OH', 'OO']
The Origin airports covered are:  ['DFW', 'LGA', 'MSN', 'MSP', 'ATL', 'BDL', 'VLD', 'JFK', 'RDU', 'CHS', 'DTW', 'GRB', 'PVD', 'SHV', 'FNT', 'PIT', 'RIC', 'RST', 'RSW', 'CVG', 'LIT', 'ORD', 'JAX', 'TRI', 'BOS', 'CWA', 'DCA', 'CHO', 'AVP', 'IND', 'GRR', 'BTR', 'MEM', 'TUL', 'CLE', 'STL', 'BTV', 'OMA', 'MGM', 'TVC', 'SAV', 'GSP', 'EWR', 'OAJ', 'BNA', 'MCI', 'TLH', 'ROC', 'LEX', 'PWM', 'BUF', 'AGS', 'CLT', 'GSO', 'BWI', 'SAT', 'PHL', 'TYS', 'ACK', 'DSM', 'GNV', 'AVL', 'BGR', 'MHT', 'ILM', 'MOT', 'IAH', 'SBN', 'SYR', 'ORF', 'MKE', 'XNA', 'MSY', 'PBI', 'ABE', 'HPN', 'EVV', 'ALB', 'LNK', 'AUS', 'PHF', 'CHA', 'GTR', 'BMI', 'BQK', 'CID', 'CAK', 'ATW', 'ABY', 'CAE', 'SRQ

**Pregunta**: ¿cuál es el recuento de todos los aeropuertos de origen y destino?

**Sugerencia**: puedes utilizar la función `values_count` de Pandas ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)) para encontrar los valores de cada aeropuerto mediante las columnas `Origin` y `Dest`.

In [15]:
counts = pd.DataFrame({'Origin':df_temp['Origin'].value_counts(), 'Destination':df_temp['Dest'].value_counts()})
counts

,Origin,Destination
ABE,303,303
ABI,169,169
ABQ,2077,2076
ABR,60,60
ABY,79,79
...,...,...
WRG,60,60
WYS,52,52
XNA,1004,1004
YAK,60,60


**Pregunta**: imprime los 15 principales aeropuertos de origen y destino en función del número de vuelos del conjunto de datos.

**Sugerencia**: puedes usar la función `sort_values` de Pandas ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)).

In [16]:
counts.sort_values(by=['Origin'],ascending=False).head(15)

,Origin,Destination
ATL,31525,31521
ORD,28257,28250
DFW,22802,22795
DEN,19807,19807
CLT,19655,19654
LAX,17875,17873
SFO,14332,14348
IAH,14210,14203
LGA,13850,13850
MSP,13349,13347


**Pregunta**: dada toda la información sobre un vuelo, ¿puedes predecir si se demorará?

**Respuesta**: la columna **ArrDel15** es una variable indicadora que toma el valor 1 cuando el retraso es superior a 15 minutos, 0 en caso contrario.

Podrías usar esto como una columna objetivo para el problema de clasificación.

Ahora, supón que viajas de San Francisco a Los Ángeles en un viaje de trabajo. Quieres saber si tu vuelo se retrasará, dado un conjunto de características, para poder administrar mejor tus reservas en Los Ángeles. ¿Cuántas características de este conjunto de datos sabrías antes del vuelo?

Columnas como `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay` y `DivArrDelay` contienen información sobre un retraso. Pero este retraso podría haberse producido en el origen o en el destino. Si hubiera un retraso repentino por la meteorología 10 minutos antes del aterrizaje, estos datos no serían útiles para administrar tus reservas de Los Ángeles.

Por lo tanto, para simplificar la declaración del problema, considera las siguientes columnas para predecir una demora de llegada:

`Year`, `Quarter`, `Month`, `DayofMonth`, `DayOfWeek`, `FlightDate`, `Reporting_Airline`, `Origin`, `OriginState`, `Dest`, `DestState`, `CRSDepTime`, `DepDelayMinutes`, `DepartureDelayGroups`, `Cancelled`, `Diverted`, `Distance`, `DistanceGroup`, `ArrDelay`, `ArrDelayMinutes`, `ArrDel15`, `AirTime`

También filtrarás los aeropuertos de origen y destino para que sean:
- Aeropuertos principales: ATL, ORD, DFW, DEN, CLT, LAX, IAH, PHX, SFO
- Las 5 aerolíneas principales: UA, OO, WN, AA, DL

Esto debería ayudar a reducir el tamaño de los datos en los archivos CSV que se van a combinar.

#### Combina todos los archivos CSV

**Sugerencia**:  
Primero, crea un DataFrame vacío que utilizarás para copiar tus DataFrames individuales de cada archivo. A continuación, para cada archivo de la lista `csv_files`:

1. Lee el archivo CSV en un DataFrame  
2. Filtra las columnas en función de la variable `filter_cols`

```
        columns = ['col1', 'col2']
        df_filter = df[columns]
```

3. Mantén solo los subset_vals en cada uno de los subset_cols. Usa la función `isin` de Pandas ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)) para comprobar si `val` está en la columna de DataFrame y luego elige las filas que lo incluyan.

```
        df_eg[df_eg['col1'].isin('5')]
```

4. Concatena el DataFrame con el DataFrame vacío 

In [17]:
def combine_csv(csv_files, filter_cols, subset_cols, subset_vals, file_name = '/home/ec2-user/SageMaker/project/data/FlightDelays/combined_files.csv'):
    """
    Combine csv files into one Data Frame
    csv_files: list of csv file paths
    filter_cols: list of columns to filter
    subset_cols: list of columns to subset rows
    subset_vals: list of list of values to subset rows
    """
    
    df = pd.DataFrame()
    
    for file in csv_files:
        df_temp = pd.read_csv(file)
        df_temp = df_temp[filter_cols]
        for col, val in zip(subset_cols,subset_vals):
            df_temp = df_temp[df_temp[col].isin(val)]      
        
        df = pd.concat([df, df_temp], axis=0)
    
        
    df.to_csv(file_name, index=False)
    print(f'Combined csv stored at {file_name}')

In [18]:
#cols is the list of columns to predict Arrival Delay 
cols = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate',
        'Reporting_Airline','Origin','OriginState','Dest','DestState',
        'CRSDepTime','Cancelled','Diverted','Distance','DistanceGroup',
        'ArrDelay','ArrDelayMinutes','ArrDel15','AirTime']

subset_cols = ['Origin', 'Dest', 'Reporting_Airline']

# subset_vals is a list collection of the top origin and destination airports and top 5 airlines
subset_vals = [['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'], 
               ['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'], 
               ['UA', 'OO', 'WN', 'AA', 'DL']]

Utiliza la función anterior para fusionar los diferentes archivos en un solo archivo que se pueda leer fácilmente. 

**Nota**: Esto tardará entre cinco y siete minutos en completarse.

In [ ]:
start = time.time()
combine_csv(csv_files, cols, subset_cols, subset_vals)
print(f'csv\'s merged in {round((time.time() - start)/60,2)} minutes')

#### Cargar conjunto de datos

Carga el conjunto de datos combinado.

In [ ]:
data = pd.read_csv('/home/ec2-user/SageMaker/project/data/FlightDelays/combined_files.csv')

Imprime los primeros cinco registros.

In [ ]:
data.head(5)

Estas son otras preguntas que te ayudarán a obtener más información sobre el conjunto de datos.

**Preguntas**   
1. ¿Cuántas filas y columnas tiene el conjunto de datos?   
2. ¿Cuántos años se incluyen en el conjunto de datos?   
3. ¿Cuál es el rango de fechas para el conjunto de datos?   
4. ¿Qué aerolíneas se incluyen en el conjunto de datos?   
5. ¿Qué aeropuertos de origen y destino están incluidos?

In [ ]:
print("The #rows and #columns are ", data.shape[0] , " and ", data.shape[1])
print("The years in this dataset are: ", list(data.Year.unique()))
print("The months covered in this dataset are: ", sorted(list(data.Month.unique())))
print("The date range for data is :" , min(data.FlightDate), " to ", max(data.FlightDate))
print("The airlines covered in this dataset are: ", list(data.Reporting_Airline.unique()))
print("The Origin airports covered are: ", list(data.Origin.unique()))
print("The Destination airports covered are: ", list(data.Dest.unique()))

Vamos a definir nuestra **columna de destino: is_delay** (1: si la hora de llegada se retrasa más de 15 minutos, 0: en el resto de los casos). Utiliza el método `rename` para cambiar el nombre de la columna de `ArrDel15` a `is_delay`.

**Sugerencia**: puedes usar la función `rename` de Pandas ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

Por ejemplo:
```
df.rename(columns={'col1':'column1'}, inplace=True)
```

In [ ]:
# Rename ArrDel15 to is_delay
data.rename(columns={'ArrDel15':'is_delay'}, inplace=True) 

Busca valores null en las columnas. Puedes utilizar la función `isnull()` ([documentación](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.isnull.html)).

**Sugerencia**: `isnull()` detecta si el valor particular es null o no y le da un valor booleano (True o False) en su lugar. Utiliza la función `sum (axis=0)` para resumir el número de columnas.

In [ ]:
data.isnull().sum(axis = 0)

Faltan los detalles de retraso de llegadas y el tiempo en el aire para 22 540 de las 1 658 130 filas, lo que representa el 1,3 %. Puedes eliminar o imputar estas filas. La documentación no menciona nada sobre filas que faltan.

**Sugerencia**: utiliza el operador `~` para elegir los valores que no son nulos de la salida `isnull()`.

Por ejemplo:
```
null_eg = df_eg[~df_eg['column_name'].isnull()]
```

In [ ]:
### Remove null columns
data = data[~data.is_delay.isnull()]
data.isnull().sum(axis = 0)

Obtén la hora del día en formato de 24 horas desde CRSDepTime.

In [ ]:
data['DepHourofDay'] = (data['CRSDepTime']//100)

## **La declaración del problema de ML**
- Dada una serie de características, ¿puedes predecir si un vuelo se va a retrasar más de 15 minutos?
- Debido a que la variable de destino solo toma un valor de 0 o 1, podrías utilizar un algoritmo de clasificación. 

Antes de pasar a modelar, siempre es una buena práctica examinar la distribución de características, las correlaciones, etc.
- Esto te dará una idea de cualquier falta de linealidad o patrones en los datos.
    - Modelos lineales: añaden características de potencia, exponenciales o de interacción
    - Prueba un modelo no lineal
- Desequilibrio de datos 
    - Elige métricas que no den un rendimiento sesgado del modelo (exactitud frente a AUC)
    - Utiliza funciones de pérdida ponderadas o personalizadas
- Datos que faltan
    - Haz una imputación basada en estadísticas simples: media, mediana, modo (variables numéricas), clase frecuente (variables categóricas)
    - Imputación basada en clústeres (KNN para predecir el valor de la columna)
    - Elimina columnas

## Exploración de datos

#### Comprueba el retraso de clase frente a ningún retraso

**Sugerencia**: utiliza un diagrama `groupby` ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)) con un diagrama `bar` ([documentation](https://matplotlib.org/tutorials/introductory/pyplot.html)) para trazar la frecuencia frente a la distribución de la clase.

In [ ]:
(data.groupby('is_delay').size()/len(data) ).plot(kind='bar')
plt.ylabel('Frequency')
plt.title('Distribution of classes')
plt.show()

**Pregunta**: ¿qué puedes deducir del diagrama de barras sobre la relación entre retraso y sin retraso?

**Respuesta:** el 80 % de los vuelos llegan a tiempo, pero el 20 % no. Este es un claro desequilibrio de clases.

**Preguntas**: 

- ¿Qué meses tienen más retrasos?
- ¿Qué hora del día tiene más retrasos?
- ¿Qué día de la semana tiene más retrasos?
- ¿Qué aerolínea tiene más retrasos?
- ¿Qué aeropuertos de origen y destino tienen más retrasos?
- ¿La distancia de vuelo es un factor en los retrasos?

In [ ]:
viz_columns = ['Month', 'DepHourofDay', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest']
fig, axes = plt.subplots(3, 2, figsize=(20,20), squeeze=False)
# fig.autofmt_xdate(rotation=90)

for idx, column in enumerate(viz_columns):
    ax = axes[idx//2, idx%2]
    temp = data.groupby(column)['is_delay'].value_counts(normalize=True).rename('percentage').\
    mul(100).reset_index().sort_values(column)
    sns.barplot(x=column, y="percentage", hue="is_delay", data=temp, ax=ax)
    plt.ylabel('% delay/no-delay')
    

plt.show()

In [ ]:
sns.lmplot( x="is_delay", y="Distance", data=data, fit_reg=False, hue='is_delay', legend=False)
plt.legend(loc='center')
plt.xlabel('is_delay')
plt.ylabel('Distance')
plt.show()

**Respuestas**
- Vemos que el % de retraso es alto en mayo, junio, julio y agosto (meses del 5 al 8).
- El % de retraso es más alto por la noche que durante el día. - No hay vuelos que salgan a las 3 de la madrugada.
- Domingo (1), miércoles (4), sábado (7) tienen retrasos relativamente mayores que otros días.
- La aerolínea WN tiene el mayor % de retrasos en comparación con otras.
- Al parecer, los retrasos ocurren tanto en vuelos de corta distancia como de larga distancia.
- Los vuelos con SFO como destino tienen retrasos más altos que otros.

Aunque algunas de las variables están codificadas en números, es importante considerarlas como variables categóricas, ya que no vemos una relación cuantitativa entre ellas y la variable objetivo.

### Características

Mira todas las columnas y cuáles son sus tipos específicos.

In [ ]:
data.columns

Filtrado de columnas requeridas:
- La fecha es redundante, porque tienes Year, Quarter, Month, DayofMonth y DayOfWeek para describir la fecha.
- Utiliza los códigos Origin y Dest en lugar de OriginState y DestState.
- Dado que solo estás clasificando si el vuelo se ha retrasado o no, no necesitas TotalDelayMinutes, DepDelayMinutes y ArrDelayMinutes.

Trata DepHourofDay como una variable categórica, ya que no tiene ninguna relación cuantitativa con el objetivo.
- Si tuvieras que hacer una codificación one-hot de ella, resultaría en 23 columnas más.
- Otras alternativas al manejo de variables categóricas incluyen codificación hash, codificación media regularizada y depósito de valores en buckets, entre otras.
- Aquí solo divide en buckets.

**Sugerencia**: para cambiar un tipo de columna a categoría, utiliza la función `astype` ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)).

In [ ]:
data_orig = data.copy()
data = data[[ 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay']]
categorical_columns  = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest', 'DepHourofDay', 'is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')

Para utilizar la codificación one-hot, utiliza la función `get_dummies` de Pandas para las columnas categóricas que has seleccionado arriba. A continuación, puedes concatenar esas características generadas a tu conjunto de datos original mediante el uso de la función `concat` en Pandas. Para codificar variables categóricas, también puedes usar *codificación ficticia* mediante una palabra clave `drop_first=True`. Para obtener más información sobre la codificación ficticia, consulta https://en.wikiversity.org/wiki/Dummy_variable_(statistics).

Por ejemplo:
```
pd.get_dummies(df[['column1','columns2']], drop_first=True)
```

In [ ]:
categorical_columns.remove('is_delay')
data_dummies = pd.get_dummies(data[categorical_columns], drop_first=True, dtype='int64')
data = pd.concat([data, data_dummies], axis = 1)
data.drop(categorical_columns,axis=1, inplace=True)

Comprueba la longitud del conjunto de datos y las nuevas columnas.

In [ ]:
len(data)

In [ ]:
data.columns

Ya estás listo para el entrenamiento de modelos. Antes de dividir los datos, cambia el nombre de la columna `is_delay` a `target`.

**Sugerencia**: puedes usar la función `rename` de Pandas ([documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

In [ ]:
data.rename(columns = {'is_delay':'target'}, inplace=True )